# Exploratory Data Analysis with SparkR

[**Introduction to Apache Spark with R by J. A. Dianes**](https://github.com/jadianes/spark-r-notebooks)

In this notebook we will use all the SparkSQL operation we learned before in order to explore property value reltion with other variables in the [2013 American Community Survey](http://www.census.gov/programs-surveys/acs/data/summary-file.html) dataset. The whole point of R on Spark is to introduce Spark scalability into R data analysis pipelines. With this idea in mind, we have seen how [SparkR documentation](http://spark.apache.org/docs/latest/sparkr.html) introduces data types and functions that are very similar to what we are used to when using regular R. We will combine these with [ggplot2](http://ggplot2.org) in order to explore relationships between our varaibles. We will explain what we do at every step but, if you want to go deeper into `ggplot2` for exploratory data analysis, I did this [Udacity on-line course](https://www.udacity.com/course/data-analysis-with-r--ud651) in the past and I highly recommend it! 

So let's dive into it!

## Creating a SparkSQL context and loading data

In order to explore our data, we first need to load it into a SparkSQL data frame. But first we need to init a SparkSQL context. The first thing we need to do is to set up some environment variables and library paths as follows. Remember to replace the value assigned to `SPARK_HOME` with your Spark home folder.  

In [ ]:
# Set Spark home and R libs
Sys.setenv(SPARK_HOME='/home/cluster/spark-1.5.0-bin-hadoop2.6')
.libPaths(c(file.path(Sys.getenv('SPARK_HOME'), 'R', 'lib'), .libPaths()))

Now we can load the `SparkR` library as follows.

In [ ]:
library(SparkR)

And now we can initialise the Spark context as [in the official documentation](http://spark.apache.org/docs/latest/sparkr.html#starting-up-sparkcontext-sqlcontext). In our case we are use a standalone Spark cluster with one master and seven workers. If you are running Spark in local node, use just `master='local'`. Additionally, we require a Spark package from Databricks to read CSV files (more on this in the [previous notebook](https://github.com/jadianes/spark-r-notebooks/blob/master/notebooks/nb1-spark-sql-basics/nb1-spark-sql-basics.ipynb)). 

In [ ]:
sc <- sparkR.init(master='spark://169.254.206.2:7077', sparkPackages="com.databricks:spark-csv_2.11:1.2.0")

And finally we can start the SparkSQL context as follows.

In [ ]:
sqlContext <- sparkRSQL.init(sc)

Now that we have our SparkSQL context ready, we can use it to load our CSV data into data frames. We have downloaded our [2013 American Community Survey dataset](http://www.census.gov/programs-surveys/acs/data/summary-file.html) files in [notebook 0](https://github.com/jadianes/spark-r-notebooks/tree/master/notebooks/nb0-starting-up/nb0-starting-up.ipynb), so they should be stored locally. Remember to set the right path for your data files in the first line, ours is `/nfs/data/2013-acs/ss13husa.csv`.  

In [ ]:
housing_a_file_path <- file.path('', 'nfs','data','2013-acs','ss13husa.csv')
housing_b_file_path <- file.path('', 'nfs','data','2013-acs','ss13husb.csv')

Now let's read into a SparkSQL dataframe. We need to pass four parameters in addition to the `sqlContext`:  

- The file path.  
- `header='true'` since our `csv` files have a header with the column names. 
- Indicate that we want the library to infer the schema.  
- And the source type (the Databricks package in this case). 

And we have two separate files for both, housing and population data. We need to join them.

In [ ]:
housing_a_df <- read.df(sqlContext, 
                        housing_a_file_path, 
                        header='true', 
                        source = "com.databricks.spark.csv", 
                        inferSchema='true')

In [ ]:
housing_b_df <- read.df(sqlContext, 
                        housing_b_file_path, 
                        header='true', 
                        source = "com.databricks.spark.csv", 
                        inferSchema='true')

In [ ]:
housing_df <- rbind(housing_a_df, housing_b_df)

Let's check that we have everything there by counting the files and listing a few of them.

In [ ]:
nrow(housing_df)

In [ ]:
head(housing_df)

## Exploring property value

- Box plots for categorical variables (E.g. region)
- Scatter plots or bubble plots for two or three numerical ordinal variables

As a starting point, let's have a look at average property values by region.

In [ ]:
housing_avg_agg <- agg(
        groupBy(housing_df, "REGION", "BDSP"), 
        NUM_PROPERTIES=n(housing_df$REGION),
        AVG_VALP = avg(housing_df$VALP), 
        MAX_VALUE=max(housing_df$VALP),
        MIN_VALUE=min(housing_df$VALP)
    )
housing_avg_sorted <- head(arrange(housing_avg_agg, desc(housing_avg_agg$AVG_VALP)))

housing_avg_sorted$REGION <- factor(
    housing_avg_sorted$REGION, 
    levels=c(1,2,3,4,9), 
    labels=c('Northeast', 'Midwest','South','West','Puerto Rico')
)
housing_avg_sorted

Let's do the same by state.

Bar plot using identity